In [1]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

from orc.utils import generate_problem
from orc.primal import *
from orc.lagrangean import subgrad_opt

In [2]:
A, b = generate_problem(20, 50)

In [36]:
X0 = np.array([False] * A.shape[-1])
X1 = np.array([False] * A.shape[-1])

In [3]:
x = hall_hochbaum(A, b)
ub = np.sum(A, axis=0) @ x
ub, x

(35201.0,
 array([0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
        0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.]))

In [4]:
lambd, lb, x_lb = subgrad_opt(A, b, ub, X0, X1)
lambd, lb, x_lb

(array([0.00000000e+00, 1.58857126e+00, 2.21734209e+00, 2.80137002e+00,
        7.58417458e-01, 9.01132058e-01, 1.02372648e+00, 1.28478068e+00,
        0.00000000e+00, 7.82259108e-01, 0.00000000e+00, 6.48971262e-01,
        5.12145340e-05, 0.00000000e+00, 4.80860242e-01, 4.65932134e+00,
        4.28830947e+00, 0.00000000e+00, 9.55872138e-03, 7.38694122e-01]),
 29653.476311837752,
 array([0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0]))

In [5]:
list(zip(A @ x_lb - b, lambd))

[(423.0, 0.0),
 (18.0, 1.588571262949864),
 (124.0, 2.217342090667555),
 (161.0, 2.8013700162233253),
 (173.0, 0.7584174575524396),
 (27.0, 0.9011320577253875),
 (220.0, 1.023726476377005),
 (196.0, 1.2847806837260642),
 (14.0, 0.0),
 (-178.0, 0.7822591083791471),
 (424.0, 0.0),
 (156.0, 0.6489712616617597),
 (27.0, 5.1214533987482e-05),
 (18.0, 0.0),
 (95.0, 0.4808602415873594),
 (-50.0, 4.659321337776286),
 (146.0, 4.288309471918593),
 (660.0, 0.0),
 (339.0, 0.009558721378360108),
 (125.0, 0.7386941217318727)]

In [6]:
list(zip((1 - lambd) @ A, x_lb))

[(249.5538849660793, 0),
 (238.86555252012155, 0),
 (793.368041389345, 0),
 (-119.29607363876326, 1),
 (-3.3719683187621, 1),
 (208.56445546112639, 0),
 (144.65434881351246, 0),
 (41.71920207993452, 0),
 (715.3692542613909, 0),
 (-47.979448683897886, 1),
 (96.27094786350966, 0),
 (-3.89676010634804, 1),
 (522.9837796534687, 0),
 (-21.256278849278942, 1),
 (265.67704725711206, 0),
 (-2.107364704172369, 1),
 (-1.027431808707961, 1),
 (32.42258628569515, 0),
 (-195.40312940889243, 1),
 (-273.1059918994456, 1),
 (350.6500202276678, 0),
 (543.203181106637, 0),
 (372.3345178109753, 0),
 (463.8137912278041, 0),
 (-333.29923546472196, 1),
 (49.476085181315455, 0),
 (204.5100201206812, 0),
 (-595.4125310583927, 1),
 (-20.65941243554105, 1),
 (49.34741988915488, 0),
 (-1.7048148543278216, 1),
 (-0.0044346598411948435, 1),
 (-2.222798391248091, 1),
 (375.5327510467439, 0),
 (55.98912769001822, 0),
 (0.05834358277041929, 0),
 (-59.31677089002159, 1),
 (411.93459649385346, 0),
 (958.8356506645366, 

In [7]:
np.sum(A, axis=0)

array([3113., 1940., 1998., 1739., 1517., 2260., 1939., 2618., 2101.,
       1834., 1939., 2229., 2787., 2481., 2296., 2033., 1479., 1200.,
       1601., 1670., 1764., 1573., 2093., 2024., 1681., 2003., 2549.,
       1977., 1193., 1437., 1791., 1597., 1140., 2098., 2445., 1687.,
       1515., 2684., 2232., 1381., 1579., 2315., 2463., 1605., 2014.,
       1585., 1875., 2455., 2292., 2256.])

In [8]:
m = gp.Model("mip")
x = m.addMVar(A.shape[-1], vtype=GRB.BINARY, name="x")
m.setObjective(np.sum(A, axis=0) @ x)
m.addConstr(A @ x >= b)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16


<MConstr (20,) *awaiting model update*>

In [9]:
m.optimize()
for v in m.getVars():
    print(v.varName, v.x)
m.getObjective().getValue()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 50 columns and 500 nonzeros
Model fingerprint: 0x674c213e
Variable types: 0 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [8e+00, 4e+02]
  Objective range  [1e+03, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 2e+03]
Found heuristic solution: objective 43354.000000
Presolve time: 0.05s
Presolved: 20 rows, 50 columns, 500 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 40511.000000

Root relaxation: objective 2.976410e+04, 23 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 29764.09

31630.0

In [18]:
r = np.argmax(b - A @ x_lb)
r

9

In [59]:
fixed0 = set(np.where(X0)[0])
fixed1 = set(np.where(X1)[0])
zero_entry = set(np.where(A[r] == 0)[0])
not_candidates = list(fixed0.union(fixed1).union(zero_entry))
not_candidates

[4,
 5,
 7,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 23,
 25,
 27,
 28,
 29,
 31,
 36,
 38,
 43,
 45,
 46]

In [57]:
cr = (1 - lambd) @ A
cr

array([ 2.49553885e+02,  2.38865553e+02,  7.93368041e+02, -1.19296074e+02,
       -3.37196832e+00,  2.08564455e+02,  1.44654349e+02,  4.17192021e+01,
        7.15369254e+02, -4.79794487e+01,  9.62709479e+01, -3.89676011e+00,
        5.22983780e+02, -2.12562788e+01,  2.65677047e+02, -2.10736470e+00,
       -1.02743181e+00,  3.24225863e+01, -1.95403129e+02, -2.73105992e+02,
        3.50650020e+02,  5.43203181e+02,  3.72334518e+02,  4.63813791e+02,
       -3.33299235e+02,  4.94760852e+01,  2.04510020e+02, -5.95412531e+02,
       -2.06594124e+01,  4.93474199e+01, -1.70481485e+00, -4.43465984e-03,
       -2.22279839e+00,  3.75532751e+02,  5.59891277e+01,  5.83435828e-02,
       -5.93167709e+01,  4.11934596e+02,  9.58835651e+02,  2.42208489e+02,
        2.63337886e+00,  1.12456107e+00,  1.25258122e+03, -5.44020471e+00,
        3.57828408e+02,  2.94109728e+02,  9.59588019e+01, -8.55965558e+01,
        8.45392933e+01,  9.14260237e+02])

In [61]:
cr[not_candidates] = np.inf
cr

array([ 2.49553885e+02,  2.38865553e+02,  7.93368041e+02, -1.19296074e+02,
                   inf,             inf,  1.44654349e+02,             inf,
        7.15369254e+02,             inf,             inf,             inf,
        5.22983780e+02,             inf,             inf,             inf,
                   inf,             inf,             inf,             inf,
                   inf,  5.43203181e+02,  3.72334518e+02,             inf,
       -3.33299235e+02,             inf,  2.04510020e+02,             inf,
                   inf,             inf, -1.70481485e+00,             inf,
       -2.22279839e+00,  3.75532751e+02,  5.59891277e+01,  5.83435828e-02,
                   inf,  4.11934596e+02,             inf,  2.42208489e+02,
        2.63337886e+00,  1.12456107e+00,  1.25258122e+03,             inf,
        3.57828408e+02,             inf,             inf, -8.55965558e+01,
        8.45392933e+01,  9.14260237e+02])

In [63]:
j = np.argmin(cr)
j
# add j to X0 in a branch and to X1 in the other

24